In [43]:
!pip install pandas
!pip install gensim


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
import re

In [45]:
 # Load the dataset
print("Loading dataset...")

df = pd.read_csv(r"C:\Users\vidya\Topic Modelling_3020\Thematic_course_outlines.csv")
#C:\Users\vidya\Topic Modelling_3020\ThematicJobs_TFIDF.csv

# Convert Important_Words column (comma-separated string) into a list
df["cleaned_description"] = df["cleaned_description"].astype(str).apply(lambda x: x.split(", "))
documents = df["cleaned_description"].tolist()



Loading dataset...


In [46]:
# Function to perform topic modeling on a given set of documents
def perform_topic_modeling(documents, num_topics=10):
    # Create a dictionary and corpus
    dictionary = corpora.Dictionary(documents)
    dictionary.filter_extremes(no_below= 0.5, no_above=0.95)
    corpus = [dictionary.doc2bow(doc) for doc in documents]
    
    # Train LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42, passes=50)
    
    return lda_model, corpus, dictionary

In [47]:
# Function to display topics with documents
def display_topics_with_docs(model, corpus, documents, dictionary, num_words=10, num_top_docs=3):
    for topic_idx in range(model.num_topics):
        print(f"\n### Topic {topic_idx}:")
        
        # Extract top words (fix: only use word IDs, not probability scores)
        top_words = [dictionary[word_id] for word_id, _ in model.get_topic_terms(topic_idx, topn=num_words)]
        print("Top Words:", " ".join(top_words))
        print("\nTop Documents:")

        # Get documents with their scores for the current topic
        topic_doc_scores = []
        for doc_idx, doc in enumerate(corpus):
            doc_topics = model.get_document_topics(doc, minimum_probability=0)  # Get topic distribution
            score = dict(doc_topics).get(topic_idx, 0)  # Extract probability for this topic
            topic_doc_scores.append((doc_idx, score))

        # Sort documents by their probability of belonging to the topic
        top_doc_indices = sorted(topic_doc_scores, key=lambda x: x[1], reverse=True)[:num_top_docs]                                                                                   
           # Print top documents
        for doc_index, score in top_doc_indices:
            print(f"Document {doc_index} (Score: {score:.4f}):")
            print(" ".join(documents[doc_index]))
            print("-" * 80)

In [48]:
# Group the dataframe by 'Thematic' and perform topic modeling for each group
for thematic, group in df.groupby('Thematic'):
    print(f"\nPerforming topic modeling for Thematic: {thematic}")
    documents = group["cleaned_description"].tolist()
    
    # Perform topic modeling
    lda_model, corpus, dictionary = perform_topic_modeling(documents)
    
    # Display topics with documents
    display_topics_with_docs(lda_model, corpus, documents, dictionary)


Performing topic modeling for Thematic: communications

### Topic 0:
Top Words: multivariable calculus probability electrical and computer engineering probability and statistic discrete mathematic analysis multivariable field theory graph theory algorithms random variable

Top Documents:
Document 7 (Score: 0.9809):
vector calculus heat transfer system analysis discrete mathematic discrete mathematic discrete mathematic experimental datum electrical and computer engineering electrical and computer engineering electrical and computer engineering computer engineering computer engineering computer engineering multivariable calculus multivariable calculus multivariable calculus multivariable calculus multivariable calculus coordinate system recurrence relation graph theory graph theory random variable probability and statistic probability and statistic probability and statistic electromagnetic field theory field theory acoustics graphs probability probability probability probability model 

In [49]:
# Function to display topics alone (without documents)
def display_topics(model, dictionary, num_words=10):
    for topic_idx in range(model.num_topics):
        # Extract top words for the topic
        top_words = [dictionary[word_id] for word_id, _ in model.get_topic_terms(topic_idx, topn=num_words)]
        print(f"Topic {topic_idx}:", " ".join(top_words))

# Group the dataframe by 'Thematic' and perform topic modeling for each group
for thematic, group in df.groupby('Thematic'):
    print(f"\nPerforming topic modeling for Thematic: {thematic}")
    documents = group["cleaned_description"].tolist()
    
    # Debugging: Check if documents are empty
    if not documents or all(len(doc) == 0 for doc in documents):
        print(f"Skipping thematic '{thematic}' because it contains no valid documents.")
        continue
    
    try:
        # Perform topic modeling
        lda_model, corpus, dictionary = perform_topic_modeling(documents)
        
        # Display topics alone
        print("\nTopics for Thematic:", thematic)
        display_topics(lda_model, dictionary)
        
    except ValueError as e:
        print(f"Error processing thematic '{thematic}': {e}") 


Performing topic modeling for Thematic: communications

Topics for Thematic: communications
Topic 0: multivariable calculus probability electrical and computer engineering probability and statistic discrete mathematic analysis multivariable field theory graph theory algorithms random variable
Topic 1: simulation electrical modulation signal processing prototype learning outcomes b embed system communication system matlab
Topic 2: signal processing electrical learning outcomes operation fft fourier transform embed system engineering problems b digital signal
Topic 3: operation management electrical act analytical computer network datum link communication system communication transport layer
Topic 4: prototype embed system communication networks electrical indicators software processes vulnerability learning outcomes b product process
Topic 5: presentations electrical presentation english engineering problems communication communicate professional technical information social
Topic 6: p